In [1]:
from meteostream import HycomClient

In [2]:
client = HycomClient()

In [3]:
client.check_server_alignment()

Forecast times are identical but are incomplete: ['sst', 'ssu', 'ssv']


False

In [20]:
from siphon.catalog import TDSCatalog
import xarray as xr
import pandas as pd

SST_URL: str = "https://tds.hycom.org/thredds/catalog/FMRC_ESPC-D-V02_t3z/runs/catalog.xml"

In [108]:
cat = TDSCatalog(SST_URL)
best_ds = cat.datasets[0]

ds_str = str(best_ds)
time_str = ds_str[-20:]
timestamp = pd.to_datetime(time_str)
timestamp = timestamp.to_datetime64()

In [109]:
ds = xr.open_dataset(best_ds.access_urls['OPENDAP'], decode_times=False) 
ds

<xarray.Dataset> Size: 141GB
Dimensions:      (depth: 40, lat: 4251, lon: 4500, time: 46, time_1: 1)
Coordinates:
  * depth        (depth) float64 320B 0.0 2.0 4.0 6.0 ... 3e+03 4e+03 5e+03
  * lat          (lat) float64 34kB -80.0 -79.96 -79.92 ... 89.92 89.96 90.0
  * lon          (lon) float64 36kB 0.0 0.07996 0.16 0.24 ... 359.8 359.8 359.9
  * time         (time) float64 368B 216.0 219.0 222.0 ... 345.0 348.0 351.0
Dimensions without coordinates: time_1
Data variables:
    tau          (time_1) float64 8B ...
    time_offset  (time) float64 368B ...
    water_temp   (time, depth, lat, lon) float32 141GB ...
Attributes: (12/22)
    classification_level:      
    distribution_statement:    Approved for public release; distribution unli...
    downgrade_date:            not applicable
    classification_authority:  not applicable
    institution:               Fleet Numerical Meteorology and Oceanography C...
    source:                    HYCOM archive file, GLBz0.04
    ...                        ...
    time_origin:               2025-01-09 12:00:00
    _CoordSysBuilder:          ucar.nc2.dataset.conv.CF1Convention
    cdm_data_type:             GRID
    featureType:               GRID
    location:                  Proto fmrc:FMRC_ESPC-D-V02_t3z
    history:                   FMRC Run 2025-01-11T12:00:00Z Dataset

In [110]:
time_offset = pd.to_timedelta(ds['time_offset'], unit='hours')
time_offset = pd.TimedeltaIndex(time_offset)
time_offset

TimedeltaIndex(['0 days 00:00:00', '0 days 03:00:00', '0 days 06:00:00',
                '0 days 09:00:00', '0 days 12:00:00', '0 days 15:00:00',
                '0 days 18:00:00', '0 days 21:00:00', '1 days 00:00:00',
                '1 days 03:00:00', '1 days 06:00:00', '1 days 09:00:00',
                '1 days 12:00:00', '1 days 15:00:00', '1 days 18:00:00',
                '1 days 21:00:00', '2 days 00:00:00', '2 days 03:00:00',
                '2 days 06:00:00', '2 days 09:00:00', '2 days 12:00:00',
                '2 days 15:00:00', '2 days 18:00:00', '2 days 21:00:00',
                '3 days 00:00:00', '3 days 03:00:00', '3 days 06:00:00',
                '3 days 09:00:00', '3 days 12:00:00', '3 days 15:00:00',
                '3 days 18:00:00', '3 days 21:00:00', '4 days 00:00:00',
                '4 days 03:00:00', '4 days 06:00:00', '4 days 09:00:00',
                '4 days 12:00:00', '4 days 15:00:00', '4 days 18:00:00',
                '4 days 21:00:00', '5 days 00:00:00

In [111]:
ds['valid_time'] = timestamp + time_offset
ds 

<xarray.Dataset> Size: 141GB
Dimensions:      (depth: 40, lat: 4251, lon: 4500, time: 46, time_1: 1,
                  valid_time: 46)
Coordinates:
  * depth        (depth) float64 320B 0.0 2.0 4.0 6.0 ... 3e+03 4e+03 5e+03
  * lat          (lat) float64 34kB -80.0 -79.96 -79.92 ... 89.92 89.96 90.0
  * lon          (lon) float64 36kB 0.0 0.07996 0.16 0.24 ... 359.8 359.8 359.9
  * time         (time) float64 368B 216.0 219.0 222.0 ... 345.0 348.0 351.0
  * valid_time   (valid_time) datetime64[ns] 368B 2025-01-11T12:00:00 ... 202...
Dimensions without coordinates: time_1
Data variables:
    tau          (time_1) float64 8B ...
    time_offset  (time) float64 368B 0.0 3.0 6.0 9.0 ... 126.0 129.0 132.0 135.0
    water_temp   (time, depth, lat, lon) float32 141GB ...
Attributes: (12/22)
    classification_level:      
    distribution_statement:    Approved for public release; distribution unli...
    downgrade_date:            not applicable
    classification_authority:  not applicable
    institution:               Fleet Numerical Meteorology and Oceanography C...
    source:                    HYCOM archive file, GLBz0.04
    ...                        ...
    time_origin:               2025-01-09 12:00:00
    _CoordSysBuilder:          ucar.nc2.dataset.conv.CF1Convention
    cdm_data_type:             GRID
    featureType:               GRID
    location:                  Proto fmrc:FMRC_ESPC-D-V02_t3z
    history:                   FMRC Run 2025-01-11T12:00:00Z Dataset

In [84]:
ds['new_time'] = pd_datetime + time_offset 

In [85]:
ds 

<xarray.Dataset> Size: 141GB
Dimensions:      (depth: 40, lat: 4251, lon: 4500, time: 46, time_1: 1,
                  valid_time: 46, new_time: 46)
Coordinates:
  * depth        (depth) float64 320B 0.0 2.0 4.0 6.0 ... 3e+03 4e+03 5e+03
  * lat          (lat) float64 34kB -80.0 -79.96 -79.92 ... 89.92 89.96 90.0
  * lon          (lon) float64 36kB 0.0 0.07996 0.16 0.24 ... 359.8 359.8 359.9
  * time         (time) float64 368B 216.0 219.0 222.0 ... 345.0 348.0 351.0
  * valid_time   (valid_time) timedelta64[ns] 368B 00:00:00 ... 5 days 15:00:00
  * new_time     (new_time) object 368B 1736596800000000000 ... 1737082800000...
Dimensions without coordinates: time_1
Data variables:
    tau          (time_1) float64 8B ...
    time_offset  (time) float64 368B 0.0 3.0 6.0 9.0 ... 126.0 129.0 132.0 135.0
    water_temp   (time, depth, lat, lon) float32 141GB ...
    ref_time     object 8B 2025-01-11T12:00:00+00:00
Attributes: (12/22)
    classification_level:      
    distribution_statement:    Approved for public release; distribution unli...
    downgrade_date:            not applicable
    classification_authority:  not applicable
    institution:               Fleet Numerical Meteorology and Oceanography C...
    source:                    HYCOM archive file, GLBz0.04
    ...                        ...
    time_origin:               2025-01-09 12:00:00
    _CoordSysBuilder:          ucar.nc2.dataset.conv.CF1Convention
    cdm_data_type:             GRID
    featureType:               GRID
    location:                  Proto fmrc:FMRC_ESPC-D-V02_t3z
    history:                   FMRC Run 2025-01-11T12:00:00Z Dataset

In [ ]:
ds = ds.set_coords("time_offset")

In [50]:
ds = ds.swap_dims({'time':'time_offset'})

In [51]:
ds = ds.drop_vars(['tau','time']) 

In [52]:
ds = ds.assign_coords(ref_time = pd_datetime)
ds = ds.rename_vars({"time_offset":"valid_time"})

In [53]:
ds['ref_time'].attrs['long_name'] = "Forecast Run"

In [54]:
ds 

<xarray.Dataset> Size: 141GB
Dimensions:     (depth: 40, lat: 4251, lon: 4500, time_offset: 46)
Coordinates:
  * depth       (depth) float64 320B 0.0 2.0 4.0 6.0 ... 3e+03 4e+03 5e+03
  * lat         (lat) float64 34kB -80.0 -79.96 -79.92 ... 89.92 89.96 90.0
  * lon         (lon) float64 36kB 0.0 0.07996 0.16 0.24 ... 359.8 359.8 359.9
  * valid_time  (time_offset) float64 368B 0.0 3.0 6.0 9.0 ... 129.0 132.0 135.0
    ref_time    object 8B 2025-01-11T12:00:00+00:00
Dimensions without coordinates: time_offset
Data variables:
    water_temp  (time_offset, depth, lat, lon) float32 141GB ...
Attributes: (12/22)
    classification_level:      
    distribution_statement:    Approved for public release; distribution unli...
    downgrade_date:            not applicable
    classification_authority:  not applicable
    institution:               Fleet Numerical Meteorology and Oceanography C...
    source:                    HYCOM archive file, GLBz0.04
    ...                        ...
    time_origin:               2025-01-09 12:00:00
    _CoordSysBuilder:          ucar.nc2.dataset.conv.CF1Convention
    cdm_data_type:             GRID
    featureType:               GRID
    location:                  Proto fmrc:FMRC_ESPC-D-V02_t3z
    history:                   FMRC Run 2025-01-11T12:00:00Z Dataset